# Problema 5: Rutas Óptimas para Equipos de Inspección de infraestrutura en Colombia
## 1. Conjuntos
$V$ para los nodos/localidades a visitar
## 2. Parámetros
$C$, Una matriz con los costos las localidades. Cada lugar i,j indica el costo entre el nodo i y nodo j
## 3. Variables de decisión
 $X_{ij}$ binaria, donde 1 quiere decir que el camino entre i y j fue seleccionado como parte del camino.
## 4. Función objetivo
$min\sum_{i\in V}^{}\sum_{j\in V}^{} c_{ij}x_{ij}$
## 5. Restricciones
#### Que todos los nodos j sean seleccionados
$\sum_{j\in V}^{}x_{ij}\forall i\in V$
#### Que todos los nodos i sean seleccionados
$\sum_{i\in V}^{}x_{ij}\forall j\in V$
## 6. Tipo de problema
MIP, pues 

In [ ]:

from __future__ import division
from pyomo.environ import *
import pandas as pd

from pyomo.opt import SolverFactory

import sys
import os

os.system("clear")

def leer_matriz_csv(file_name):
    df = pd.read_csv(file_name,header=None)
    num_nodos = df.shape[0]

    matriz_costos = {}
    for i in range(num_nodos):
        for j in range(num_nodos):
            if i != j:
                matriz_costos[(i,j)] = df.iloc[i,j]
            else:
                matriz_costos[(i,j)] = 999 

Model = ConcreteModel()

# SETS & PARAMETERS********************************************************************
numNodes=5

N=RangeSet(1, numNodes)

cost={(1,1):999, (1,2):5,   (1,3):2,   (1,4):999, (1,5):999,\
      (2,1):999, (2,2):999, (2,3):999, (2,4):999, (2,5):8,\
      (3,1):999, (3,2):999, (3,3):999, (3,4):3,   (3,5):999,\
      (4,1):999, (4,2):999, (4,3):999, (4,4):999, (4,5):2,\
      (5,1):999, (5,2):999, (5,3):999, (5,4):999, (5,5):999}

# VARIABLES****************************************************************************
Model.x = Var(N,N, domain=Binary)

# OBJECTIVE FUNCTION*******************************************************************
Model.obj = Objective(expr = sum(Model.x[i,j]*cost[i,j] for i in N for j in N))

# CONSTRAINTS**************************************************************************
Model.res1=ConstraintList()
for i in N:
    if i==1:
        Model.res1.add(sum(Model.x[i,j] for j in N)==1)

Model.res2=ConstraintList()
for j in N:
    if j==5:
        Model.res2.add(sum(Model.x[i,j] for i in N)==1)

Model.res3=ConstraintList()
for i in N:
    if i!=1 and i!=5:
        Model.res3.add(sum(Model.x[i,j] for j in N) - sum(Model.x[j,i] for j in N)==0)
    
# APPLYING THE SOLVER******************************************************************
SolverFactory('glpk').solve(Model)

Model.display()



